<a href="https://colab.research.google.com/github/MLBartkowiakChojnacki/ML_PROJECT_2022/blob/grid-search/notebooks/loading_preprocessing_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import files
src = list(files.upload().values())[0]
open('handling_outliers.py','wb').write(src)

Saving handling_outliers.py to handling_outliers.py


1844

In [81]:
#imports
import pandas as pd
import pprint
import numpy as np
from handling_outliers import removing_iqr, removing_percentiles, zscore_outlier, modified_z_score_outlier, count_outliers, mask_outliers, replace_missing_values
from sklearn.preprocessing import RobustScaler, binarize
from sklearn.dummy import DummyClassifier
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV, RepeatedStratifiedKFold, cross_val_score
from sklearn.metrics import confusion_matrix, balanced_accuracy_score, f1_score, precision_score, recall_score, balanced_accuracy_score, roc_auc_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

In [75]:
#loading
X = pd.read_csv('/content/drive/MyDrive/train_data.csv', header = None)
y = pd.read_csv('/content/drive/MyDrive/train_labels.csv', header = None)

In [ ]:
#outliers
X_rem = removing_iqr(pd.DataFrame(X))
counted = count_outliers(X_rem)
contamination = counted[0].sum() / (X_rem.shape[0] * X_rem.shape[1]) * 100
masked = mask_outliers(pd.DataFrame(X), X_rem)
replaced = replace_missing_values(masked, 5)
print(counted)
print(f'Dataset contamination: {round(contamination, 2)} %')
print('_'*10)

(0       82
1       77
2       51
3       62
4       80
        ..
3745    68
3746    65
3747    79
3748    78
3749    68
Length: 3750, dtype: int64,            0
3667  0.0105
2844  0.0099
2524  0.0098
2820  0.0097
3458  0.0096
...      ...
3204  0.0048
7     0.0048
909   0.0047
3594  0.0046
345   0.0046

[3750 rows x 1 columns])
Dataset contamination: 0.71 %
__________


In [ ]:
#baseline
methods = ["most_frequent", "prior", "stratified", "uniform"]
scores = {}

for method in methods:
  dummy_clf = DummyClassifier(strategy=method)
  dummy_clf.fit(X, y)
  y_pred = dummy_clf.predict(X)
  precision = precision_score(y, y_pred)
  recall = recall_score(y, y_pred)
  acc = balanced_accuracy_score(y, y_pred)
  f1 = f1_score(y, y_pred, average='weighted')
  roc_score = roc_auc_score(y, y_pred, average='weighted')
  scores[method] = {'acc':dummy_clf.score(X, y), 
                    'precision':precision, 
                    'recall':recall, 
                    'accuracy_balanced':acc, 
                    'f1_weighted':f1, 
                    'roc_score':roc_score}

pprint.pprint(scores)

{'most_frequent': {'acc': 0.9,
                   'accuracy_balanced': 0.5,
                   'f1_weighted': 0.8526315789473684,
                   'precision': 0.9,
                   'recall': 1.0,
                   'roc_score': 0.5},
 'prior': {'acc': 0.9,
           'accuracy_balanced': 0.5,
           'f1_weighted': 0.8526315789473684,
           'precision': 0.9,
           'recall': 1.0,
           'roc_score': 0.5},
 'stratified': {'acc': 0.8186666666666667,
                'accuracy_balanced': 0.4992592592592593,
                'f1_weighted': 0.821849011553793,
                'precision': 0.8998527245949927,
                'recall': 0.9051851851851852,
                'roc_score': 0.4992592592592593},
 'uniform': {'acc': 0.532,
             'accuracy_balanced': 0.49303703703703705,
             'f1_weighted': 0.5908392354179768,
             'precision': 0.8974771873322598,
             'recall': 0.4954074074074074,
             'roc_score': 0.4930370370370371}}


In [74]:
scaler = RobustScaler()
X = scaler.fit_transform(X)

In [76]:
#encoding
y = pd.DataFrame(binarize(y))

In [77]:
#dimensionality reduction
pca = PCA(n_components=0.95, whiten=True)
X = pca.fit_transform(X)
X.shape

(3750, 1012)

In [93]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

pipe = Pipeline(steps=[("classifier", LogisticRegression())])

In [71]:
# model_params = [
                # {"classifier" : [LogisticRegression()],
                #  "classifier__solver" : ['sag', 'saga', 'liblinear'],
                # "classifier__penalty" : ['l1', 'l2', 'elasticnet', 'none'],
                # "classifier__C" : C_value,
                # "classifier__class_weight" : ['balanced', {"0" : 0.1, "1" : 0.9}]}
                # ,
                # {"classifier" : [RandomForestClassifier()],
                # "classifier__n_estimators" : [10, 100, 1000],
                # "classifier__criterion" : ['gini', 'entropy', 'log_loss'],
                # "classifier__max_features" : ['auto', 'sqrt', 'log2', 'none'],
                # "classifier__class_weight" : ['balanced', [{0 : 1}, {1 : 9}]]},
                # {"classifier":[KNeighborsClassifier()],
                # "classifier__n_neighbors":[3,5,7,9,11,13,15,17,19,21],
                #  "classifier__weights": ['uniform', 'distance']}
                # ]

model_params = [{"classifier" : [LogisticRegression()],
                 "classifier__solver" : ['sag', 'saga', 'liblinear'],
                "classifier__penalty" : ['l1', 'l2', 'elasticnet', 'none'],
                "classifier__C" : [np.logspace(1, 4, 10)],
                "classifier__class_weight" : ['balanced', {"0" : 0.1, "1" : 0.9}]},
                {"classifier" : [BernoulliNB()],
                "classifier__alpha" : [np.linspace(1, 10, 100)]},
                {"classifier" : [RandomForestClassifier()],
                "classifier__n_estimators" : [10, 100, 1000],
                "classifier__criterion" : ['gini', 'entropy', 'log_loss'],
                "classifier__max_features" : ['auto', 'sqrt', 'log2', 'none'],
                "classifier__class_weight" : ['balanced', [{0 : 1}, {1 : 9}]]},
                {"classifier" : [KNeighborsClassifier()],
                "classifier__weights" : ['uniform', 'distance'],
                "classifier__algorithm" : ['auto', 'ball-tree', 'kd-tree', 'brute'],
                "classifier__n_neighbors" : [np.linspace(1, 11, 6)]},
                {"classifier" : [SVC()],
                "classifier__C" : [np.logspace(1, 10000, 1000)],
                "classifier__class_weight" : ['balanced', [{0 : 1}, {1 : 9}]]},
                {"classifier" : [MLPClassifier()]},
                {"classifier" : [DecisionTreeClassifier()],
                "classifier__criterion" : ['gini', 'entropy', 'log_loss'],
                "classifier__splitter" : ['best', 'random'],
                "classifier__max_features" : ['auto', 'sqrt', 'log2', 'none'],
                "classifier__class_weight" : ['balanced', [{0 : 1}, {1 : 9}]]},
                {"classifier" : [ExtraTreesClassifier()],
                "classifier__n_estimators" : [10, 100, 1000],
                "classifier__criterion" : ['gini', 'entropy', 'log_loss'],
                "classifier__max_features" : ['sqrt', 'log2', 'none'],
                "classifier__class_weight" : ['balanced', [{0 : 1}, {1 : 9}]]}]

/usr/local/lib/python3.7/dist-packages/numpy/core/function_base.py:277: RuntimeWarning: overflow encountered in power
  return _nx.power(base, y)


In [94]:
scores = []

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, stratify=y, shuffle=True)

search = GridSearchCV(pipe, model_params, scoring="f1_weighted", refit=True, verbose=3)

In [95]:
result = search.fit(X_train, y_train.values.flatten())

Fitting 5 folds for each of 210 candidates, totalling 1050 fits
[CV 1/5] END classifier=LogisticRegression(), classifier__C=[   10.            21.5443469     46.41588834   100.
   215.443469     464.15888336  1000.          2154.43469003
  4641.58883361 10000.        ], classifier__class_weight=balanced, classifier__penalty=l1, classifier__solver=sag;, score=nan total time=   0.0s
[CV 2/5] END classifier=LogisticRegression(), classifier__C=[   10.            21.5443469     46.41588834   100.
   215.443469     464.15888336  1000.          2154.43469003
  4641.58883361 10000.        ], classifier__class_weight=balanced, classifier__penalty=l1, classifier__solver=sag;, score=nan total time=   0.0s
[CV 3/5] END classifier=LogisticRegression(), classifier__C=[   10.            21.5443469     46.41588834   100.
   215.443469     464.15888336  1000.          2154.43469003
  4641.58883361 10000.        ], classifier__class_weight=balanced, classifier__penalty=l1, classifier__solver=sag;, score

KeyboardInterrupt: ignored